In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
import glob

# only case1

In [ ]:
fs = glob.glob('out0*.nc')
fs.sort()

In [ ]:
for nn, f in enumerate(fs):
    ds = xr.open_dataset(f)
    U = ds['u'].values
    V = ds['v'].values
    Vmag = np.sqrt( U**2 + V**2) + 0.00000001
    angle = (np.pi/2.) - np.arctan2(U/Vmag, V/Vmag)
    
    dx, dy = 1.0, 1.0
    omega = np.zeros_like(U)
    for i in range(1, ds.dims['x']-1):
        for j in range(1, ds.dims['y']-1):
            omega[i,j] = (V[i+1,j] - V[i-1,j])/2.0/dx - (U[i,j+1] - U[i,j-1])/2.0/dy 
    
    dss = xr.Dataset( { 'Vmag': (['x','y'], Vmag), 'angle': (['x','y'], angle), 'omega': (['x','y'], omega) }
                       , coords={ 'x':ds['x'] , 'y':ds['y']}
                    ) 
    
    # vstep = 10 #間引き間隔
    dssp = dss.isel(x=slice(0,dss.dims['x'],20), y=slice(0,dss.dims['y'],5))
    
    figVec = dssp.hvplot.vectorfield(frame_width=1400, frame_height=100, x='x', y='y', angle='angle', mag='Vmag',hover=False).opts(magnitude='Vmag', scale=0.2, color='k')
    
    f = (dss['Vmag'].hvplot(x='x', y='y', cmap='reds', clim=(0,4.2), title='velocity')*figVec \
     + dss['omega'].hvplot(x='x', y='y', cmap='bwr',frame_width=1400, frame_height=100, clim=(-0.5,0.5), title='vorticity')).cols(1)
    
    fo = f.options(title=str(int(ds.total_second))+'sec')
    out = hvplot.save(fo, 'out' + str(nn).zfill(8) + '.html')
    
del out

# case1 and case2

In [ ]:
def makefig(f,ncase):
    ds = xr.open_dataset(f)
    U = ds['u'].values
    V = ds['v'].values
    Vmag = np.sqrt( U**2 + V**2) + 0.00000001
    angle = (np.pi/2.) - np.arctan2(U/Vmag, V/Vmag)
    
    dx, dy = 1.0, 1.0
    omega = np.zeros_like(U)
    for i in range(1, ds.dims['x']-1):
        for j in range(1, ds.dims['y']-1):
            omega[i,j] = (V[i+1,j] - V[i-1,j])/2.0/dx - (U[i,j+1] - U[i,j-1])/2.0/dy 
    
    dss = xr.Dataset( { 'Vmag': (['x','y'], Vmag), 'angle': (['x','y'], angle), 'omega': (['x','y'], omega) }
                       , coords={ 'x':ds['x'] , 'y':ds['y']}
                    ) 
    
    # vstep = 10 #間引き間隔
    dssp = dss.isel(x=slice(0,dss.dims['x'],20), y=slice(0,dss.dims['y'],5))
    
    figVec = dssp.hvplot.vectorfield(frame_width=1400, frame_height=100, x='x', y='y', angle='angle', mag='Vmag',hover=False).opts(magnitude='Vmag', scale=0.2, color='k')
    
#     f = (dss['Vmag'].hvplot(x='x', y='y', cmap='reds', clim=(0,4.2), title='velocity')*figVec \
#      + dss['omega'].hvplot(x='x', y='y', cmap='bwr',frame_width=1400, frame_height=100, clim=(-0.5,0.5), title='vorticity')).cols(1)
    
    f1 = dss['Vmag'].hvplot(x='x', y='y', cmap='reds', clim=(0,4.2), title=ncase + ':velocity')*figVec
    f2 = dss['omega'].hvplot(x='x', y='y', cmap='bwr',frame_width=1400, frame_height=100, clim=(-0.5,0.5), title=ncase +':vorticity')
    
    return f1, f2, ds.total_second

In [ ]:
# fs = glob.glob('friction_const/out0*.nc')
fs1 = glob.glob('case1_zeroEq/out0*.nc')
fs1.sort()
fs2 = glob.glob('out0*.nc')
fs2.sort()

nn = 0
for f1, f2 in zip(fs1, fs2):
    f1vel, f1vor, time = makefig(f1, 'case1')
    f2vel, f2vor, time = makefig(f2, 'case2')
    f = (f1vel + f1vor + f2vel + f2vor).cols(1)
    fo = f.options(title=str(int(time))+'sec')
    out = hvplot.save(fo, 'out' + str(nn).zfill(8) + '.html')
    nn += 1
    
del out

# manning

In [ ]:
fs1 = glob.glob('case1_zeroEq/out0*.nc')
f = fs1[0]

ds = xr.open_dataset(f)
cManning = np.full_like(ds['u'].values, 0.025, dtype=np.float64)
cManning[:,:30] = 0.1

ds["manning"] = xr.DataArray(cManning, coords=[('x',ds['x']),('y',ds['y'])])

In [ ]:
fig = ds['manning'].hvplot(x='x', y='y', cmap='bwr',frame_width=1400, frame_height=100)
#                             , clim=(-0.5,0.5), title=ncase +':vorticity')

In [ ]:
hvplot.save(fig, 'manning.png')